# Лабораторная работа №3 Проведение исследований с решающим деревом

Подключим необходимые базовые модули

In [ ]:
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, root_mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer

Загрузим датасеты

In [ ]:
kagglehub.dataset_download("yasserh/wine-quality-dataset")
kagglehub.dataset_download("shree1992/housedata")

100%|██████████| 21.5k/21.5k [00:00<00:00, 27.7MB/s]

Extracting files...


100%|██████████| 432k/432k [00:00<00:00, 798kB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2'

In [ ]:
df_class = pd.read_csv("/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv")
df_class = df_class.drop(columns=['Id'])
y_class = df_class['quality']
X_class = df_class.drop(columns=['quality'])
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=7575, shuffle = True
)

In [ ]:
df_reg = pd.read_csv("/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2/data.csv")
df_reg = df_reg.drop(columns=['street', 'country', 'date'])

categorical_features = ['city', 'statezip']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

encoder.fit(df_reg[categorical_features])
encoded_cols = encoder.get_feature_names_out(categorical_features)
encoded_df = pd.DataFrame(encoder.transform(df_reg[categorical_features]), columns=encoded_cols, index=df_reg.index)

df_reg = pd.concat([df_reg.drop(columns=categorical_features), encoded_df], axis=1)

y_reg = df_reg['price']
X_reg = df_reg.drop(columns=['price'])

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=4668, shuffle=True
)

## 2. Создание бейзлайна и оценка качества

Для начала подключим и обучим модель из *sklearn*

In [ ]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

In [ ]:
classifier = DecisionTreeClassifier(random_state=6346)
classifier.fit(X_train_class, y_train_class)

DecisionTreeClassifier(random_state=6346)

In [ ]:
regressor = DecisionTreeRegressor(random_state=75356)
regressor.fit(X_train_reg, y_train_reg)

DecisionTreeRegressor(random_state=75356)

Проведем оценку полученных моделей

In [ ]:
y_pred_class = classifier.predict(X_test_class)
y_pred_reg = regressor.predict(X_test_reg)

accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

rmse_reg = root_mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print("Classification - Decision Tree:")
print(f"Accuracy: {accuracy_class:.4f}")
print(f"F1 Score: {f1_class:.4f}\n")

print("Regression - Decision Tree:")
print(f"RMSE: {rmse_reg:.4f}")
print(f"R^2 Score: {r2_reg:.4f}")

Classification - Decision Tree:
Accuracy: 0.5459
F1 Score: 0.5416

Regression - Decision Tree:
RMSE: 777331.9901
R^2 Score: 0.0550


Как можно заметить решающее дерево показало себя очень хорошо даже на базовом бейзлане для задачи классификации, что не требует анализа признаков и дает приемлимую точность.
Для задачи регрессии модель показла достаточно высокую ошибка в RMSE, возможно дальнейшее улучшение бейзлана поможет улучшить ситуацию

## 3. Улучшение бейзлайна и оценка качества

Я решил использовать улучшенный бейзлайн из прошлой лабораторной работы для задачи регрессии, т.к. он показал себя достаточно хорошо
Для задачи классификации я решил сравнить бейзлайн из первой и второй лабораторной работы, чтобы понять, как upsampling влияет на точность

Итоговые гипотезы которые проверю в данной лабораторной работе:
- Классификация-1
  1. Попробуем убрать residual sugar и pH из-за слабой корреляции
  2. Примением логарифмическое преобразование к признакам с сильным перекосом
- Классификация-2
  1. Убрать признаки residual sugar, pH
  2. Прологарифмируем признаки chlorides и sulphates
  3. Создадим признак-отношение Free Sulfur Dioxide и Total Sulfur Dioxide
  4. Попробуем расширить классы quality с помощью oversampling
- Регрессия
  1. Попробуем прологарифмировать целевую переменную price т.к. она имеет большой разбросом и данные в основном смещены вправо
  2. Удалить признак sqft_lot, condition из-за низкой корреляции
  3. Попробуем перевести признак waterfront в бинарный
  4. Обрежем выбросы для bedrooms и bathrooms

### Проверим гипотезы обучив новые модели

Сначала сравним модели для классификации, обучим первый датасет без использования Oversampling

In [ ]:
df_class = pd.read_csv('/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv')

df_class = df_class.drop(columns=['residual sugar', 'pH'])

df_class['total sulfur dioxide'] = np.log1p(df_class['total sulfur dioxide'])
df_class['free sulfur dioxide'] = np.log1p(df_class['free sulfur dioxide'])

X_class = df_class.drop(columns=['quality', 'Id'])
y_class = df_class['quality']

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.3, random_state=5435, shuffle=True
)

Обучим первое решающее дерево

In [ ]:
classifier_improve_one = DecisionTreeClassifier(random_state=6346)
classifier_improve_one.fit(X_train_class, y_train_class)

DecisionTreeClassifier(random_state=6346)

In [ ]:
y_pred_class = classifier_improve_one.predict(X_test_class)

accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

print("Classification - Decision Tree Improve:")
print(f"Accuracy: {accuracy_class:.4f}")
print(f"F1 Score: {f1_class:.4f}\n")

Classification - Decision Tree Improve:
Accuracy: 0.5977
F1 Score: 0.5976



Результаты сильно улучшились, по сравнению с прошлой лабораторной работой

Теперь сравним с бейзлайном в котором используется SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

df_class = pd.read_csv("/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv")
df_class = df_class.drop(columns=['Id'])

df_class = df_class.drop(columns=['residual sugar', 'pH'])

df_class['chlorides'] = np.log1p(df_class['chlorides'])
df_class['sulphates'] = np.log1p(df_class['sulphates'])

df_class['sulfur_ratio'] = df_class['free sulfur dioxide'] / df_class['total sulfur dioxide']

y_class = df_class['quality']
X_class = df_class.drop(columns=['quality'])

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.3, random_state=5435, shuffle=True
)

# Для oversampling используем SMOTE
smote = SMOTE(random_state=5353, k_neighbors=4)
X_train_class, y_train_class = smote.fit_resample(X_train_class, y_train_class)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


In [ ]:
classifier_improve_two = DecisionTreeClassifier(random_state=6346)
classifier_improve_two.fit(X_train_class, y_train_class)

DecisionTreeClassifier(random_state=6346)

In [ ]:
y_pred_class = classifier_improve_two.predict(X_test_class)

accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

print("Classification - Decision Tree Improve Two:")
print(f"Accuracy: {accuracy_class:.4f}")
print(f"F1 Score: {f1_class:.4f}\n")

Classification - Decision Tree Improve Two:
Accuracy: 0.5190
F1 Score: 0.5342



Как можем заметить точность решения упала. Скорее всего это связано с использованием SMOTE. В дальнейшем я принял решение отойти от использования этого метода

Теперь обучим модель для регрессии

In [ ]:
df_reg = pd.read_csv("/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2/data.csv")
df_reg = df_reg.drop(columns=['street', 'country', 'date'])

df_reg['price'] = np.log1p(df_reg['price'])

df_reg = df_reg.drop(columns=['sqft_lot', 'condition'])
df_reg['waterfront'] = df_reg['waterfront'].apply(lambda x: 1 if x > 0 else 0)

bedroom_upper_limit = df_reg['bedrooms'].quantile(0.95)
bathroom_upper_limit = df_reg['bathrooms'].quantile(0.95)

df_reg = df_reg[
    (df_reg['bedrooms'] <= bedroom_upper_limit) &
    (df_reg['bathrooms'] <= bathroom_upper_limit)
]

categorical_features = ['city', 'statezip']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder.fit(df_reg[categorical_features])
encoded_cols = encoder.get_feature_names_out(categorical_features)
encoded_df = pd.DataFrame(encoder.transform(df_reg[categorical_features]), columns=encoded_cols, index=df_reg.index)
df_reg = pd.concat([df_reg.drop(columns=categorical_features), encoded_df], axis=1)

y_reg = df_reg['price']
X_reg = df_reg.drop(columns=['price'])

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=54356, shuffle=True
)

In [ ]:
regressor_improve = DecisionTreeRegressor(random_state=15860)
regressor_improve.fit(X_train_reg, y_train_reg)

DecisionTreeRegressor(random_state=15860)

In [ ]:
reg_preds = regressor_improve.predict(X_test_reg)
reg_preds_exp = np.expm1(reg_preds)
y_test_reg_exp = np.expm1(y_test_reg)

rmse_reg = root_mean_squared_error(y_test_reg_exp, reg_preds_exp)
r2_reg = r2_score(y_test_reg_exp, reg_preds_exp)

print("Regression - Decision Tree:")
print(f"RMSE: {rmse_reg:.4f}")
print(f"R^2 Score: {r2_reg:.4f}")

Regression - Decision Tree:
RMSE: 238610.6618
R^2 Score: 0.4482


Аналогично с задачей классификацией метрики модели для регрессии сильно улучшились

### Сравнение бейзлайнов

- Классификация

|          | Первый baseline | Улучшенный baseline |
|----------|-----------------|---------------------|
| Accuracy | 0.5459          | 0.5977             |
| F1       | 0.5416         | 0.5976              |

- Регрессия

|       | Первый baseline | Улучшенный baseline |
|-------|-----------------|---------------------|
| RMSE  | 777331.99      | 238610.66           |
| $R^2$ | 0.0550         | 0.4482              |

### Выводы

Решающие деревья пока показывают себя как быстрый и надежный способ решения задачи. С улучшенным бейзлайном модель показала достаточно высокие результаты.

## Имплементация алгоритма машинного обучения

Имплементируем собственное решающее дерево в виде одного класса, который будем параметризовать для задачи классификации и для задачи регрессии

In [ ]:
class DecisionTreeModel:
    def __init__(self, mode="classification", max_levels=None):
        self.mode = mode
        self.max_levels = max_levels
        self.root = None

    def train(self, features, labels):
        self.root = self._grow_tree(features, labels, current_depth=0)

    def predict(self, features):
        return np.array([self._navigate_tree(sample, self.root) for sample in features])

    def _grow_tree(self, features, labels, current_depth):
        if self.max_levels is not None and current_depth >= self.max_levels:
            return self._create_terminal_node(labels)

        if len(set(labels)) == 1:
            return self._create_terminal_node(labels)

        split_feature, split_value = self._identify_optimal_split(features, labels)
        if split_feature is None:
            return self._create_terminal_node(labels)

        left_indices = features[:, split_feature] <= split_value
        right_indices = features[:, split_feature] > split_value

        left_branch = self._grow_tree(features[left_indices], labels[left_indices], current_depth + 1)
        right_branch = self._grow_tree(features[right_indices], labels[right_indices], current_depth + 1)

        return {
            "split_feature": split_feature,
            "split_value": split_value,
            "left": left_branch,
            "right": right_branch
        }

    def _identify_optimal_split(self, features, labels):
        optimal_feature, optimal_value = None, None
        best_metric = float("inf") if self.mode == "regression" else -float("inf")

        for feature_idx in range(features.shape[1]):
            unique_values = np.unique(features[:, feature_idx])
            for value in unique_values:
                left_mask = features[:, feature_idx] <= value
                right_mask = features[:, feature_idx] > value

                if sum(left_mask) == 0 or sum(right_mask) == 0:
                    continue

                metric = self._evaluate_split(labels, left_mask, right_mask)

                if (self.mode == "regression" and metric < best_metric) or (
                    self.mode == "classification" and metric > best_metric
                ):
                    best_metric = metric
                    optimal_feature = feature_idx
                    optimal_value = value

        return optimal_feature, optimal_value

    def _evaluate_split(self, labels, left_mask, right_mask):
        if self.mode == "regression":
            left_error = np.var(labels[left_mask]) * sum(left_mask)
            right_error = np.var(labels[right_mask]) * sum(right_mask)
            return left_error + right_error
        else:
            left_score = self._compute_gini(labels[left_mask]) * sum(left_mask)
            right_score = self._compute_gini(labels[right_mask]) * sum(right_mask)
            return -(left_score + right_score)

    def _compute_gini(self, labels):
        proportions = np.bincount(labels) / len(labels)
        return 1 - np.sum(proportions ** 2)

    def _create_terminal_node(self, labels):
        if self.mode == "regression":
            return np.mean(labels)
        else:
            return np.bincount(labels).argmax()

    def _navigate_tree(self, sample, node):
        if not isinstance(node, dict):
            return node

        if sample[node["split_feature"]] <= node["split_value"]:
            return self._navigate_tree(sample, node["left"])
        else:
            return self._navigate_tree(sample, node["right"])

Проверим теперь реализованную модель в задачах

Загрузим изначальные датасеты, без применения feature engineering

In [ ]:
df_class = pd.read_csv("/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv")
df_class = df_class.drop(columns=['Id'])
y_class = df_class['quality']
X_class = df_class.drop(columns=['quality'])
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=7575, shuffle = True
)

In [ ]:
df_reg = pd.read_csv("/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2/data.csv")
df_reg = df_reg.drop(columns=['street', 'country', 'date'])

categorical_features = ['city', 'statezip']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

encoder.fit(df_reg[categorical_features])
encoded_cols = encoder.get_feature_names_out(categorical_features)
encoded_df = pd.DataFrame(encoder.transform(df_reg[categorical_features]), columns=encoded_cols, index=df_reg.index)

df_reg = pd.concat([df_reg.drop(columns=categorical_features), encoded_df], axis=1)

y_reg = df_reg['price']
X_reg = df_reg.drop(columns=['price'])

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=4668, shuffle=True
)

Обучим наши модели

In [ ]:
classifier_impl = DecisionTreeModel(mode="classification", max_levels=5)
classifier_impl.train(X_train_class.to_numpy(), y_train_class.to_numpy())

In [ ]:
regressor_impl = DecisionTreeModel(mode="regression", max_levels=5)
regressor_impl.train(X_train_reg.to_numpy(), y_train_reg.to_numpy())

Оценим метрики полученных моделей

In [ ]:
y_pred_class = classifier_impl.predict(X_test_class.to_numpy())
y_pred_reg = regressor_impl.predict(X_test_reg.to_numpy())

accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

rmse_reg = root_mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print("Classification - Decision Tree Impl:")
print(f"Accuracy: {accuracy_class:.4f}")
print(f"F1 Score: {f1_class:.4f}\n")

print("Regression - Decision Tree Impl:")
print(f"RMSE: {rmse_reg:.4f}")
print(f"R^2 Score: {r2_reg:.4f}")

Classification - Decision Tree Impl:
Accuracy: 0.5371
F1 Score: 0.5248

Regression - Decision Tree Impl:
RMSE: 761764.2501
R^2 Score: 0.0925


- Классификация

| Logistic | Библиотека      | Имплементация       |
|----------|-----------------|---------------------|
| Accuracy | 0.5459          | 0.5371               |
| F1       | 0.5416          | 0.5248              |

- Регрессия

| Linear | Библиотека      | Имплиментация    |
|--------|-----------------|------------------|
| RMSE   | 777331.99       | 761764.25        |
| $R^2$  | 0.0550          | 0.0925           |


Полученные метрики получились чуть хуже, чем у библиотечных для решения задачи классификации, но лучше для чем для задачи регрессии.

### Обучим модель на улучшенном бейзлайне

Подготовим датасет

In [ ]:
df_class = pd.read_csv('/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv')

df_class = df_class.drop(columns=['residual sugar', 'pH'])

df_class['total sulfur dioxide'] = np.log1p(df_class['total sulfur dioxide'])
df_class['free sulfur dioxide'] = np.log1p(df_class['free sulfur dioxide'])

X_class = df_class.drop(columns=['quality', 'Id'])
y_class = df_class['quality']

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.3, random_state=5435, shuffle=True
)

In [ ]:
df_reg = pd.read_csv("/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2/data.csv")
df_reg = df_reg.drop(columns=['street', 'country', 'date'])

df_reg['price'] = np.log1p(df_reg['price'])

df_reg = df_reg.drop(columns=['sqft_lot', 'condition'])
df_reg['waterfront'] = df_reg['waterfront'].apply(lambda x: 1 if x > 0 else 0)

bedroom_upper_limit = df_reg['bedrooms'].quantile(0.95)
bathroom_upper_limit = df_reg['bathrooms'].quantile(0.95)

df_reg = df_reg[
    (df_reg['bedrooms'] <= bedroom_upper_limit) &
    (df_reg['bathrooms'] <= bathroom_upper_limit)
]

categorical_features = ['city', 'statezip']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder.fit(df_reg[categorical_features])
encoded_cols = encoder.get_feature_names_out(categorical_features)
encoded_df = pd.DataFrame(encoder.transform(df_reg[categorical_features]), columns=encoded_cols, index=df_reg.index)
df_reg = pd.concat([df_reg.drop(columns=categorical_features), encoded_df], axis=1)

y_reg = df_reg['price']
X_reg = df_reg.drop(columns=['price'])

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=54356, shuffle=True
)

Обучим наши модели

In [ ]:
classifier_impl_improve = DecisionTreeModel(mode="classification", max_levels=5)
classifier_impl_improve.train(X_train_class.to_numpy(), y_train_class.to_numpy())

In [ ]:
regressor_impl_improve = DecisionTreeModel(mode="regression", max_levels=5)
regressor_impl_improve.train(X_train_reg.to_numpy(), y_train_reg.to_numpy())

Получим метрики для наших моделей

In [ ]:
y_pred_class = classifier_impl_improve.predict(X_test_class.to_numpy())
y_pred_reg = regressor_impl_improve.predict(X_test_reg.to_numpy())
y_pred_reg_exp = np.expm1(y_pred_reg)
y_test_reg_exp = np.expm1(y_test_reg)

accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

rmse_reg = root_mean_squared_error(y_test_reg_exp, y_pred_reg_exp)
r2_reg = r2_score(y_test_reg_exp, y_pred_reg_exp)

print("Classification - Decision Tree Impl Improve:")
print(f"Accuracy: {accuracy_class:.4f}")
print(f"F1 Score: {f1_class:.4f}\n")

print("Regression - Decision Tree Impl Improve:")
print(f"RMSE: {rmse_reg:.4f}")
print(f"R^2 Score: {r2_reg:.4f}")

Classification - Decision Tree Impl Improve:
Accuracy: 0.5714
F1 Score: 0.5612

Regression - Decision Tree Impl Improve:
RMSE: 262317.2312
R^2 Score: 0.3332


Метрики показали хороший рост точности для модели классификации и существенный рост для модели регрессии

### Сравним результаты работы библиотечной и имплментированной функции

- Классификация

|          | Библиотека      | Имплементация |
|----------|-----------------|---------------|
| Accuracy | 0.5977          |  0.5714       |
| F1       | 0.5976          |  0.5612       |

- Регрессия

|       | Библиотека      | Имплементация    |
|-------|-----------------|------------------|
| RMSE  | 238610.66       | 262317.23        |
| $R^2$ | 0.4482          | 0.3332           |

В итоге реализованная модель показала себя чуть хуже, чем библиотечная.

## Выводы

В данной лабораторной работе было исследовано и реализовано решающее дерево. Метод показал себя как простой в использовании и на "сырых" данных показал наиболее высокие результаты, что позволяет его использовать его в первом приближении решении задачи. Также в задаче были исследована два подхода к улучшению бейзлана, в результате чего было принято решение отказаться от использования метода upsampling  